PHYS 1321
Midterm Project

Collaborators: Sarah Motz, Joe Albro, Fernando Tabares

In [1]:
#imports

import numpy as np
from time import sleep
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from IPython import display # manipulate notebook output
from matplotlib.animation import FuncAnimation # animate by repeatedly calling a function
import ipywidgets as widgets

In [2]:
#define a fit function (used later in the code) for our curve

def fit_func(x, a, b, c):
    return a*x**2 + b*x +c

In [3]:
def deployment_calculation(current_velocity,current_altitude, current_time, A, B, C):
    "Calculates the time of deployment using a linear shooting method."
    #Declare some constants
    g       = -9.8
    #Choosing a very small time increment to perform my calculation. This was arbitrarily set.
    delta_t =  0.1 
    #b is a summarized way of setting all the constants for my rocket, 
    #e.g. mass, coefficient of drag, frontal surface area, and air pressure of the environment.
    b       = -8.2e-5 
    
    starting_velocity = current_velocity #each time we go through the Main loop, we want the deployment calculation to start at the current velocity and altitude
    starting_altitude = current_altitude
    
    time_to_test = current_time #I first will test whether the airbrake needs to be deployed right now
    
    projected_altitude = 800 #place-holder
    
    
    while abs(projected_altitude - 700) >= 5 : #Target altitude is 700m, with +/-5m tolerance.
        
        y0 = starting_altitude
        v0 = starting_velocity
        vf = 100 #place-hoder
        
        if starting_altitude > 690 :
            break
        
        time_to_test = time_to_test + 0.5 
        
        
        
        #The following while loop performed the shooting method. Based on the altitude and velocity at a particular moment, it calculates altitude and velocity at the next moment
        
        while abs(vf) >= 2.0:  #We want this while loop to stop when we achieve a vf close to zero, because that indicates we have reached apogee.
                        
            yf = y0 + v0 * delta_t + (1/2)  *(g + b*v0**2) * delta_t**2
            vf = v0 + (g + b * v0**2)*delta_t
            
            y0 = yf
            v0 = vf
                        
        
        
        #The work of this iteration of the loop is done. When the iteration ends, the apogee is compared with our desired apogee.
        
        #The rest of the code in the loop just modifies our variables to prepare for the next iteration of the loop
        
        starting_altitude = A*(time_to_test)**2 + B*time_to_test + C 
        starting_velocity = 2 * A * (time_to_test) + B
        
    
        projected_altitude = yf #Set the projected altitude to the particular apogee we calculated
    
       
    deployment_time = time_to_test
    print("Predicted Deployment Time = ", deployment_time)
        
    return deployment_time

In [4]:
def predictTraj(fit_func, time, altitude, current_time):
    #Fitting our flight data to a parabola
    params = curve_fit(fit_func, time[87:], altitude[87:]) 
    
    #extracting the coefficients of that parabola
    [A, B, C] = params[0] 
    
    #the time at which our projected parabola reaches its max
    t_root = -B / (2*A) 
    
    #Finding maximum point in our parabola by plugging in t_root
    max_alt = A*t_root**2 + B * t_root + C 
    #finding current velocity
    current_velocity = 2*A*current_time + B
    
    return max_alt, current_velocity, A, B, C

In [5]:
def main():
    
    #First we extract all the data from our file
    all_times,all_altitudes = np.genfromtxt("flightdata.txt",skip_header=5,unpack=True,usecols=(0,2)) #altitude in meters
    
    #Giving place-holder values to some important quantities
    deploy_time = np.nan
    time = np.array([])
    altitude = np.array([])
    prev_max_alt = 0
    deployment_time = 1000
    
    
    # values to be used in plotting
    #denoted with a p_
    #brake is set to false. This might be misleading. It is to say that the target height has been reached or not.
    p_time = []
    p_curr = []
    p_pred = []
    brake = False
    p_brake = []
    p_t_pred = []
    prediction_time = 0
    
    
    for i in range(len(all_times)):
        
        #sleep(50e-3) # Wait to simulate real time data aquisition.
        #The for loop is meant to simulate the frequency with which we will gather data during flight.
        
        time = np.append(time, all_times[i] ) # Update arrays with new data point
        
        
        current_time = time[-1] # Current data point is the last entry in updated arrays
        #set the break target if the deployment time (ie time we reach target height) is smaller than the current time
        if current_time >= deployment_time: 
            
            brake = True
            
        #simulate the taking of data by providing one more point of altitude.
        altitude = np.append(altitude, all_altitudes[i] )
        #current altitude is the final value from the array of altitude values
        current_altitude = altitude[-1]
        
        if current_time >= 5.250: # Don't want to do calculations while motor is still burning
            #We know the motor will stop burning after 3.35 seconds
            #but we also don't wanna do any calculations unless we have at least four data points.
            #For that reason, although the motor burns out at 3.350 seconds, we will start graphing at 3.55 seconds
            
            
            #set the current time to be plotted along with the current altitude
            p_time.append(time)
            p_curr.append(altitude)
            
            #fit a function to the data, and from that find the maximum altitude and current velocity
            max_alt, current_velocity, A, B, C = predictTraj(
                                                                fit_func, 
                                                                time, 
                                                                altitude, 
                                                                current_time)
            
            if max_alt > 700:
                #only want to calculate the values if the current time is less than 6 seconds into the launch
                #and if we have not reached the final altitude 
                if current_time < 6 and brake == False:
                    #calculate the time at which we will reach the target height using some magical methods
                    #magic = linear shooting
                    deployment_time = deployment_calculation(
                                                                current_velocity, 
                                                                current_altitude, 
                                                                current_time, 
                                                                A, 
                                                                B, 
                                                                C) #Fernando's function
                    #give a value that the prediction time happened.
                    prediction_time = current_time
                    
            #append the value of brake
            p_brake.append(brake)
            #create an array that we can use to calculate the predicted trajectory
            t_pred = np.linspace(current_time, 29.05, 582-i)
            #append the array to an array of arrays
            p_t_pred.append(t_pred)
            #append the predicted trajectory to an array of arrays
            p_pred.append(A*t_pred**2+B*t_pred+C)
            
        else:
            #in this case we are not going to predict anything since the motor is burning
            p_time.append(time)
            p_curr.append(altitude)
            #empty arrays so when we go to plot these values they dont show up as anything
            p_t_pred.append([])
            p_pred.append([])
            #set if the braking altitude is reached yet
            p_brake.append(brake)
    #return values so we can plot them next.
    return p_time, p_brake, p_curr, p_pred, p_t_pred, deployment_time, prediction_time



In [6]:
#running our function and collecting the returned values in some variables

p_time, p_brake, p_curr, p_pred, p_t_pred, deployment_time, prediction_time = main()

Predicted Deployment Time =  9.25
Predicted Deployment Time =  8.8
Predicted Deployment Time =  8.85
Predicted Deployment Time =  8.4
Predicted Deployment Time =  8.45


In [7]:
fig = plt.figure(figsize= (8, 6), dpi = 100)

#create some empty plots for the values that we want to graph

lines = plt.plot([], "b-", [], "r--", [], "c^", [], "go", [], "k^", [], "m*")

#assign some names to the plots
#current trajectory
ct = lines[0]
#predicted trajectory
pt = lines[1]
#target altitude (calculated)
ta = lines[2]
#burn time
burn = lines[3]
#brake light i.e. brake deployment
bl = lines[4]
#prediction time
pd = lines[5]

#give the graphs some labels
ct.set_label("Current Trajectory")
pt.set_label("Projected Trajectory")
ta.set_label("Target Altitude")
burn.set_label("Burn Finished")
bl.set_label("Brake Deployment")
pd.set_label("Time of Prediction")
#make the prediction time bigger to highlight what we actually sought to do
pd.set_markersize(20)

#print out the deployment time
print("Deployment Time = ", round(deployment_time-0.5, 3))
#set some limits on the graph based on what we know to be max altitude and the total amount of time
plt.ylim(0, 900)
plt.xlim(0, 30)
#add axis labels 
plt.xlabel("Time (s)")
plt.ylabel("Altitude (m)")
#give the bad boi a title
plt.title("Real Time Prediction of Trajectory")
#show the legend
plt.legend()
#define an animation function
def animate(frame, p_time, p_brake, p_curr, p_pred, p_t_pred, deployment_time, prediction_time):
    #set up some x and y values for current and predicted trajectories
    ctx = p_time[frame]
    cty = p_curr[frame]
    ptx = p_t_pred[frame]
    pty = p_pred[frame]
    #set the data for the current trajectory
    ct.set_data((ctx, cty))
    #set the data for the predicted trajectory
    pt.set_data((ptx, pty))
    #set the target altitude marker
    if p_brake[frame] == True and ta.get_xdata().size == 0:
        ta.set_data((ctx[-1], cty[-1]))
    #set the burn finished marker
    if (frame*50)/1000 >= 5.250 and burn.get_xdata().size == 0:
        burn.set_data((ctx[-1], cty[-1]))
    #set the brake light height
    if frame*50/1000 >= deployment_time-0.5 and bl.get_xdata().size == 0:
        bl.set_data((ctx[-1], cty[-1]))
    #set the predicted time marker
    if frame*50/1000 >= prediction_time and pd.get_xdata().size == 0:
        pd.set_data((ctx[-1], cty[-1]))
#boiler plate code with some modification: fargs to pass in arguments to animate, and frames = 582.
#interval is 50 because data is taken in 50 ms increments
anim = FuncAnimation(fig, animate, frames = 582, fargs = (p_time, p_brake, p_curr, p_pred, p_t_pred, deployment_time, prediction_time), interval = 50)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()


Deployment Time =  7.95
